In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
#import pandas as pd    
import os

os.chdir("/content/drive/MyDrive/DeepLearningCourseHW/project/") 
#h5 = pd.HDFStore('coco_detections.hdf5', 'r')

import h5py
f = h5py.File('coco_detections.hdf5', 'r')
list(f.keys())

KeyboardInterrupt: ignored

In [7]:
list(f.keys())


['100000_boxes',
 '100000_cls_prob',
 '100000_features',
 '100001_boxes',
 '100001_cls_prob',
 '100001_features',
 '100006_boxes',
 '100006_cls_prob',
 '100006_features',
 '100008_boxes',
 '100008_cls_prob',
 '100008_features',
 '100010_boxes',
 '100010_cls_prob',
 '100010_features',
 '100012_boxes',
 '100012_cls_prob',
 '100012_features',
 '100014_boxes',
 '100014_cls_prob',
 '100014_features',
 '100016_boxes',
 '100016_cls_prob',
 '100016_features',
 '100020_boxes',
 '100020_cls_prob',
 '100020_features',
 '100021_boxes',
 '100021_cls_prob',
 '100021_features',
 '100022_boxes',
 '100022_cls_prob',
 '100022_features',
 '100034_boxes',
 '100034_cls_prob',
 '100034_features',
 '100037_boxes',
 '100037_cls_prob',
 '100037_features',
 '100050_boxes',
 '100050_cls_prob',
 '100050_features',
 '10005_boxes',
 '10005_cls_prob',
 '10005_features',
 '100063_boxes',
 '100063_cls_prob',
 '100063_features',
 '100064_boxes',
 '100064_cls_prob',
 '100064_features',
 '100078_boxes',
 '100078_cls_prob

In [9]:
print(f['101456_cls_prob'])
print(f['101456_boxes'])
print(f['101456_features'])

<HDF5 dataset "101456_cls_prob": shape (48, 1601), type "<f4">
<HDF5 dataset "101456_boxes": shape (48, 4), type "<f4">
<HDF5 dataset "101456_features": shape (48, 2048), type "<f4">


In [11]:
import torch
from models.transformer import Transformer, MemoryAugmentedEncoder, MeshedDecoder, ScaledDotProductAttentionMemory
from data import COCO, DataLoader

# Pipeline for text
text_field = TextField(init_token='<bos>', eos_token='<eos>', lower=True, tokenize='spacy',
                           remove_punctuation=True, nopoints=False)
dataset = COCO(image_field, text_field, 'coco/images/', args.annotation_folder, args.annotation_folder)
    _, _, test_dataset = dataset.splits
text_field.vocab = pickle.load(open('vocab.pkl', 'rb'))

encoder = MemoryAugmentedEncoder(3, 0, attention_module=ScaledDotProductAttentionMemory,
                                     attention_module_kwargs={'m': 40})
decoder = MeshedDecoder(len(text_field.vocab), 54, 3, text_field.vocab.stoi['<pad>'])
model = Transformer(text_field.vocab.stoi['<bos>'], encoder, decoder).to(device)

data = torch.load('meshed_memory_transformer.pth')
model.load_state_dict(data['state_dict'])

IndentationError: ignored